# Cleaning Airports dataset

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from os.path import join, dirname
from dotenv import load_dotenv

## World Airports Extended dataset


The dataset world-airports-extended.csv has been created from the merged of three primary datasets, in order to simplify the process. It is included in "data" folder. The original datasets are:
- world_airports.csv: downloaded from Ourairports (https://ourairports.com/data/)
    
- airports-extended.csv: downloaded from Openfligts (https://openflights.org/data.html). Column "Tz database time zone" is taken from here.
    
- country-codes.csv: downloaded from Datahub (https://datahub.io/core/country-codes). We extract columns "ISO4217-currency_alphabetic_code" and "ISO4217-currency_name" from here.

This columns should be clean because they will be the input for different searches, thus is needed to standarise results.

In [30]:
#Get table:
airports = pd.read_csv('../data/world-airports-extended.csv')
airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,Tz database time zone,ISO4217-currency_alphabetic_code,ISO4217-currency_name
0,2434,EGLL,large_airport,London Heathrow Airport,514.706,-461.941,83.0,EU,United Kingdom,GB,...,LHR,NaN,http://www.heathrowairport.com/,https://en.wikipedia.org/wiki/Heathrow_Airport,"LON, Londres",1251675,2019-10-19T10:16:49+00:00,Europe/London,GBP,Pound Sterling
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942.501,-118.407.997,125.0,NaN,United States,US,...,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,NaN,1335475,2020-04-26T22:37:22+00:00,America/Los_Angeles,USD,US Dollar
2,3754,KORD,large_airport,Chicago O'Hare International Airport,419.786,-879.048,672.0,NaN,United States,US,...,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00,America/Chicago,USD,US Dollar
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639.801,-737.789,13.0,NaN,United States,US,...,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild",1052075,2021-04-10T17:02:47+00:00,America/New_York,USD,US Dollar
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,336.367,-84.428.101,1026.0,NaN,United States,US,...,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00,America/New_York,USD,US Dollar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64011,324519,PL-0137,small_airport,Żłobek Mały Airfield,51.454.611,23.542.472,515.0,EU,Poland,PL,...,NaN,NaN,NaN,NaN,NaN,0,2020-05-02T12:34:24+00:00,\N,PLN,Zloty
64012,42246,BITH,closed,Sauðanes Airport,66.251.859,-15.273.914,15.0,EU,Iceland,IS,...,NaN,NaN,NaN,https://en.wikipedia.org/wiki/%C3%9E%C3%B3rsh%...,"THO, Thorshofn",0,2021-03-23T00:07:10+00:00,\N,ISK,Iceland Krona
64013,313231,HOO,small_airport,Nhon Co Airfield,11.9787,1.075.638,2270.0,AS,Vietnam,VN,...,HOO,NaN,NaN,NaN,NaN,0,2021-01-10T20:43:41+00:00,NaN,VND,Dong
64014,323411,PL-0098,small_airport,Łagowiec,5.233.525,15.665.277,NaN,EU,Poland,PL,...,NaN,NaN,NaN,NaN,NaN,0,2017-05-24T02:23:15+00:00,\N,PLN,Zloty


In [32]:
#See columns
airports.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'country_name', 'iso_country',
       'region_name', 'iso_region', 'local_region', 'municipality',
       'scheduled_service', 'gps_code', 'iata_code', 'local_code', 'home_link',
       'wikipedia_link', 'keywords', 'score', 'last_updated',
       'Tz database time zone', 'ISO4217-currency_alphabetic_code',
       'ISO4217-currency_name'],
      dtype='object')

In [38]:
# Let's clean column 'Tz database time zone' first. See the data on it:
airports['Tz database time zone'].unique()


array(['Europe/London', 'America/Los_Angeles', 'America/Chicago',
       'America/New_York', 'Europe/Amsterdam', 'Europe/Paris',
       'Europe/Berlin', 'America/Denver', 'America/Phoenix',
       'America/Toronto', 'Europe/Madrid', 'Europe/Rome', 'Europe/Zurich',
       'Europe/Brussels', 'Europe/Copenhagen', 'Europe/Dublin',
       'Asia/Singapore', 'Asia/Tokyo', 'Europe/Vienna',
       'Pacific/Honolulu', 'America/Vancouver', 'Asia/Dubai',
       'Asia/Hong_Kong', 'Europe/Prague', 'Europe/Stockholm',
       'Australia/Sydney', 'Europe/Lisbon', 'Europe/Istanbul',
       'Asia/Bangkok', 'America/Cancun', 'Europe/Budapest', 'Europe/Oslo',
       'Europe/Athens', 'Asia/Shanghai', 'Europe/Helsinki',
       'Atlantic/Reykjavik', 'Asia/Kuala_Lumpur', 'America/Edmonton',
       'America/Mexico_City', 'Australia/Hobart', 'Asia/Seoul',
       'Africa/Johannesburg', 'Europe/Warsaw', 'Asia/Calcutta', '\\N',
       'America/Puerto_Rico', 'Europe/Moscow', 'Pacific/Auckland',
       'America/Sao_P

In [39]:
# We can see that there are 2 not valid values: '\\N' and nan --> TRansform them to 0
airports.loc[(airports['Tz database time zone']=='\\N'), 'Tz database time zone'] = 0

In [44]:
airports[['Tz database time zone']] = airports[['Tz database time zone']].fillna(0)

In [45]:
airports['Tz database time zone'].unique()

array(['Europe/London', 'America/Los_Angeles', 'America/Chicago',
       'America/New_York', 'Europe/Amsterdam', 'Europe/Paris',
       'Europe/Berlin', 'America/Denver', 'America/Phoenix',
       'America/Toronto', 'Europe/Madrid', 'Europe/Rome', 'Europe/Zurich',
       'Europe/Brussels', 'Europe/Copenhagen', 'Europe/Dublin',
       'Asia/Singapore', 'Asia/Tokyo', 'Europe/Vienna',
       'Pacific/Honolulu', 'America/Vancouver', 'Asia/Dubai',
       'Asia/Hong_Kong', 'Europe/Prague', 'Europe/Stockholm',
       'Australia/Sydney', 'Europe/Lisbon', 'Europe/Istanbul',
       'Asia/Bangkok', 'America/Cancun', 'Europe/Budapest', 'Europe/Oslo',
       'Europe/Athens', 'Asia/Shanghai', 'Europe/Helsinki',
       'Atlantic/Reykjavik', 'Asia/Kuala_Lumpur', 'America/Edmonton',
       'America/Mexico_City', 'Australia/Hobart', 'Asia/Seoul',
       'Africa/Johannesburg', 'Europe/Warsaw', 'Asia/Calcutta', 0,
       'America/Puerto_Rico', 'Europe/Moscow', 'Pacific/Auckland',
       'America/Sao_Paulo

In [46]:
# Let's clean column 'ISO4217-currency_alphabetic_code'. See the data on it:
airports['ISO4217-currency_alphabetic_code'].unique()

array(['GBP', 'USD', 'EUR', 'CAD', 'CHF', 'DKK', 'SGD', 'JPY', 'AED',
       'HKD', 'CZK', 'SEK', 'AUD', 'TRY', 'THB', 'MXN', 'HUF', 'NOK',
       'CNY', 'ISK', 'MYR', 'KRW', 'ZAR', 'PLN', 'INR', 'RUB', 'NZD',
       'BRL', 'ILS', 'ARS', 'EGP', 'PHP', 'TWD', 'IDR', 'PEN', 'CLP',
       'PAB,USD', 'CRC', 'VND', 'KES', 'RSD', 'QAR', 'HRK', 'RON', 'MAD',
       'COP', 'BSD', 'DOP', 'CUP,CUC', 'JMD', 'OMR', 'LKR', 'UAH', 'TND',
       'BGN', 'JOD', 'ANG', 'KHR', 'BHD', 'FJD', 'KWD', 'MVR', 'SAR',
       'ETB', 'XOF', 'NPR', 'AWG', 'VES', 'BBD', 'LBP', 'GTQ', 'UYU',
       'BAM', 'BOB', 'GHS', 'UGX', 'BMD', 'TTD', 'NGN', 'BDT', 'KYD',
       'GIP', 'MOP', 'NIO', 'GEL', 'TZS', 'SVC,USD', 'IRR', 'MKD', 'MMK',
       'SCR', 'AZN', 'PKR', 'BYN', 'NAD,ZAR', 'MUR', 'BZD', 'SDG', 'DZD',
       'KZT', 'BND', 'XCD', 'XPF', 'LAK', 'HTG,USD', 'KGS', 'AFN', 'ALL',
       'HNL', 'RWF', 'ZMW', 'LYD', 'ZWL', 'UZS', 'MDL', 'DJF', 'CDF',
       'MZN', 'AOA', 'CVE', 'SYP', 'VUV', 'GMD', 'BWP', 'LRD', 'MNT',


In [47]:
# Clean the values that have more than one code separated by comma. We keep only the first. An nan transform to 0
airports['ISO4217-currency_alphabetic_code'] = airports['ISO4217-currency_alphabetic_code'].str.split(',', expand=True)


In [49]:
airports[['ISO4217-currency_alphabetic_code']] = airports[['ISO4217-currency_alphabetic_code']].fillna(0)

In [50]:
airports['ISO4217-currency_alphabetic_code'].unique()

array(['GBP', 'USD', 'EUR', 'CAD', 'CHF', 'DKK', 'SGD', 'JPY', 'AED',
       'HKD', 'CZK', 'SEK', 'AUD', 'TRY', 'THB', 'MXN', 'HUF', 'NOK',
       'CNY', 'ISK', 'MYR', 'KRW', 'ZAR', 'PLN', 'INR', 'RUB', 'NZD',
       'BRL', 'ILS', 'ARS', 'EGP', 'PHP', 'TWD', 'IDR', 'PEN', 'CLP',
       'PAB', 'CRC', 'VND', 'KES', 'RSD', 'QAR', 'HRK', 'RON', 'MAD',
       'COP', 'BSD', 'DOP', 'CUP', 'JMD', 'OMR', 'LKR', 'UAH', 'TND',
       'BGN', 'JOD', 'ANG', 'KHR', 'BHD', 'FJD', 'KWD', 'MVR', 'SAR',
       'ETB', 'XOF', 'NPR', 'AWG', 'VES', 'BBD', 'LBP', 'GTQ', 'UYU',
       'BAM', 'BOB', 'GHS', 'UGX', 'BMD', 'TTD', 'NGN', 'BDT', 'KYD',
       'GIP', 'MOP', 'NIO', 'GEL', 'TZS', 'SVC', 'IRR', 'MKD', 'MMK',
       'SCR', 'AZN', 'PKR', 'BYN', 'NAD', 'MUR', 'BZD', 'SDG', 'DZD',
       'KZT', 'BND', 'XCD', 'XPF', 'LAK', 'HTG', 'KGS', 'AFN', 'ALL',
       'HNL', 'RWF', 'ZMW', 'LYD', 'ZWL', 'UZS', 'MDL', 'DJF', 'CDF',
       'MZN', 'AOA', 'CVE', 'SYP', 'VUV', 'GMD', 'BWP', 'LRD', 'MNT',
       'MGA', 'PYG',

In [51]:
# Let's clean column 'ISO4217-currency_name'. See the data on it:
airports['ISO4217-currency_name'].unique()

array(['Pound Sterling', 'US Dollar', 'Euro', 'Canadian Dollar',
       'Swiss Franc', 'Danish Krone', 'Singapore Dollar', 'Yen',
       'UAE Dirham', 'Hong Kong Dollar', 'Czech Koruna', 'Swedish Krona',
       'Australian Dollar', 'Turkish Lira', 'Baht', 'Mexican Peso',
       'Forint', 'Norwegian Krone', 'Yuan Renminbi', 'Iceland Krona',
       'Malaysian Ringgit', 'Won', 'Rand', 'Zloty', 'Indian Rupee',
       'Russian Ruble', 'New Zealand Dollar', 'Brazilian Real',
       'New Israeli Sheqel', 'Argentine Peso', 'Egyptian Pound',
       'Philippine Peso', 'New Taiwan dollar', 'Rupiah', 'Sol',
       'Chilean Peso', 'Balboa,US Dollar', 'Costa Rican Colon', 'Dong',
       'Kenyan Shilling', 'Serbian Dinar', 'Qatari Rial', 'Kuna',
       'Romanian Leu', 'Moroccan Dirham', 'Colombian Peso',
       'Bahamian Dollar', 'Dominican Peso', 'Cuban Peso,Peso Convertible',
       'Jamaican Dollar', 'Rial Omani', 'Sri Lanka Rupee', 'Hryvnia',
       'Tunisian Dinar', 'Bulgarian Lev', 'Jordanian D

In [52]:
# Clean the values that have more than one code separated by comma. We keep only the first. An nan transform to 0
airports['ISO4217-currency_name'] = airports['ISO4217-currency_name'].str.split(',', expand=True)

In [56]:
#Nan values transformed to 0, 'No universal currency' transformed to 0
airports[['ISO4217-currency_name']] = airports[['ISO4217-currency_name']].fillna(0)
airports.loc[(airports['ISO4217-currency_name']=='No universal currency'), 'ISO4217-currency_name'] = 0

In [57]:
airports['ISO4217-currency_name'].unique()

array(['Pound Sterling', 'US Dollar', 'Euro', 'Canadian Dollar',
       'Swiss Franc', 'Danish Krone', 'Singapore Dollar', 'Yen',
       'UAE Dirham', 'Hong Kong Dollar', 'Czech Koruna', 'Swedish Krona',
       'Australian Dollar', 'Turkish Lira', 'Baht', 'Mexican Peso',
       'Forint', 'Norwegian Krone', 'Yuan Renminbi', 'Iceland Krona',
       'Malaysian Ringgit', 'Won', 'Rand', 'Zloty', 'Indian Rupee',
       'Russian Ruble', 'New Zealand Dollar', 'Brazilian Real',
       'New Israeli Sheqel', 'Argentine Peso', 'Egyptian Pound',
       'Philippine Peso', 'New Taiwan dollar', 'Rupiah', 'Sol',
       'Chilean Peso', 'Balboa', 'Costa Rican Colon', 'Dong',
       'Kenyan Shilling', 'Serbian Dinar', 'Qatari Rial', 'Kuna',
       'Romanian Leu', 'Moroccan Dirham', 'Colombian Peso',
       'Bahamian Dollar', 'Dominican Peso', 'Cuban Peso',
       'Jamaican Dollar', 'Rial Omani', 'Sri Lanka Rupee', 'Hryvnia',
       'Tunisian Dinar', 'Bulgarian Lev', 'Jordanian Dinar',
       'Netherlands 

In [59]:
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,Tz database time zone,ISO4217-currency_alphabetic_code,ISO4217-currency_name
0,2434,EGLL,large_airport,London Heathrow Airport,514.706,-461.941,83.0,EU,United Kingdom,GB,...,LHR,NaN,http://www.heathrowairport.com/,https://en.wikipedia.org/wiki/Heathrow_Airport,"LON, Londres",1251675,2019-10-19T10:16:49+00:00,Europe/London,GBP,Pound Sterling
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942.501,-118.407.997,125.0,NaN,United States,US,...,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,NaN,1335475,2020-04-26T22:37:22+00:00,America/Los_Angeles,USD,US Dollar
2,3754,KORD,large_airport,Chicago O'Hare International Airport,419.786,-879.048,672.0,NaN,United States,US,...,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00,America/Chicago,USD,US Dollar
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639.801,-737.789,13.0,NaN,United States,US,...,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild",1052075,2021-04-10T17:02:47+00:00,America/New_York,USD,US Dollar
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,336.367,-84.428.101,1026.0,NaN,United States,US,...,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00,America/New_York,USD,US Dollar


In [61]:
#Save the dataset modificated
airports.to_csv('world-airports-extended.csv')